# 1.2 By Randomized Measurement
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import EntropyMeasure

experiment_randomized = EntropyMeasure()
# It's default method. EntropyMeasure(method='randomized') also works

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_randomized.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_randomized.add(GHZ(i), f"ghz_{i}")
    experiment_randomized.add(make_neel_circuit(i), f"neel_{i}")

experiment_randomized.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x75408012cfb0>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x753f996e1820>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x753f99e798e0>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x754081703050>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x753fa77d3170>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x753f995086e0>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x753f995220f0>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x753f99522180>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x753f99522390>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x753f99522510>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x753f99522690>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.randomized_measure.arguments import (
    EntropyMeasureRandomizedMeasureArgs,
)

Preparing a configuration list for multiple experiments with following parameters:

```python
class EntropyMeasureRandomizedMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function :func:`generate_random_unitary_seeds` 
    in :mod:`qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds

        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[EntropyMeasureRandomizedMeasureArgs] = [
    {
        "shots": 1024,
        "wave": f"{wave_names}_{i}",
        "times": 100,
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_randomized.multiOutput(
    config_list,
    summoner_name="qurrent.randomized_measure",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| Write "qurrent.randomized_measure.001", at location "qurrent.randomized_measure.001"


| 0/180   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/180   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for d747b2a9-8728-4d85-abcd-f691d9c9e023


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.001/qurryinfo.json...
| Exporting qurrent.randomized_measure.001/qurryinfo.json done.


'd747b2a9-8728-4d85-abcd-f691d9c9e023'

You can check the result of `multiOutput` that we just executed by accessing the `.multimanagers`

In [6]:
experiment_randomized.multimanagers

MultiManagerContainer(num=1, {
  "d747b2a9-8728-4d85-abcd-f691d9c9e023":
    <MultiManager(name="qurrent.randomized_measure.001", jobstype="local", ..., exps_num=180)>,
})

In [7]:
experiment_randomized.multimanagers[multi_exps1]

<MultiManager(id="d747b2a9-8728-4d85-abcd-f691d9c9e023",
  name="qurrent.randomized_measure.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-06-17 19:46:46',},
  exps_num=180)>

### d. Run post-processing at once


In [8]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 0/180 - Analysis:  - 00:00 < ?

| "report.001" has been completed.


'd747b2a9-8728-4d85-abcd-f691d9c9e023'

In [9]:
print("| Available results:")
for k, v in (
    experiment_randomized.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


- Example of the content of `quantity_container`

In [10]:
experiment_randomized.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10")
][:2]

[{'purity': np.float64(1.0204687881469727),
  'entropy': np.float64(-0.02923205706519441),
  'puritySD': np.float64(0.6447086333372172),
  'entropySD': np.float64(0.9114614370742952),
  'num_classical_registers': 10,
  'classical_registers': [0, 1],
  'classical_registers_actually': [0, 1],
  'all_system_source': 'independent',
  'purityAllSys': np.float64(1.6160114288330079),
  'entropyAllSys': np.float64(-0.6924374012223802),
  'puritySDAllSys': np.float64(0.8897919779637544),
  'entropySDAllSys': np.float64(0.7943622496272226),
  'num_classical_registers_all_sys': 10,
  'classical_registers_all_sys': None,
  'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  'errorRate': np.float64(-0.2714612029246201),
  'mitigatedPurity': np.float64(0.7265942787159235),
  'mitigatedEntropy': np.float64(0.46077808903027784),
  'counts_num': 100,
  'taking_time': 0.0005954,
  'taking_time_all_sys': 0.027875765,
  'counts_used': None,
  'input': {'num_qubits': 10,
   'selected_

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [12]:
expkeys_of_multi_exps1 = list(
    experiment_randomized.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [13]:
expkeys_of_multi_exps1

['578e611d-e9c6-44b9-9e32-f27c2844d68f',
 'ab242664-6442-41a4-ba28-0212cc1a771a',
 'c48eb11b-bea5-442b-be45-1a82d20adb6f',
 'fe53efa6-a6fd-48f9-ab93-221946407cbe',
 '844594f1-2cd9-407a-a633-45fcf8f0b7c4',
 '127fcc87-0c3e-410e-809d-c9f7455fade5',
 '1f64b79b-a257-4a4c-be17-875c0559dbec',
 '3b801d1b-865a-4736-8c03-926f7fec2cd7',
 '43bb7bcd-3f1f-445d-9935-e1b1ac25545a',
 'b85db631-8a95-4eb9-a3e3-eb6e9ac20eb1',
 'cd75de8a-d715-4469-bd7e-2c3273e03a0b',
 '45f14f26-6ae0-40bb-8207-36832f445b8e',
 '73f8ff28-9223-4141-a9df-17ae65ce7e57',
 '67aa522e-7342-4e95-9963-dfbc73db8f5c',
 'f323d7be-403b-4aa9-8f8f-831fcc9b5548',
 '393fc1c3-6d25-4eba-b3e3-b389c4001a46',
 '203ec3e5-bbba-4095-87f5-5b78d6e93371',
 'd14f37f3-ab26-4c3b-8fb6-0c71cfed01be',
 '6a023dd8-d9f5-44d7-b1d1-6408cdf4b9d9',
 '547750dd-078f-47ab-ac2f-f15100e4c40a',
 '5eed54e8-8113-4a5e-b281-3650e3361e07',
 '7442f136-8e6e-4f9f-b3c6-428195a38711',
 '880f4e8c-d975-4855-8851-cc4bcb308202',
 'e3465367-1fc4-4cf5-b49a-66dbca80f7e4',
 '65ff44f9-7a2f-

In [14]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={
        k: (
            {
                "selected_qubits": [0, 1],
            }
            if idx < 3
            else False
        )
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 0/180 - Analysis:  - 00:00 < ?

| "first_3.002" has been completed.


'ac5ddb2b-7699-4134-be7e-b154bc0dfe8c'

In [15]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [16]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={
        k: {
            "selected_qubits": [0, 1],  # selected qubits for the analysis
        }
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 0/180 - Analysis:  - 00:00 < ?

| "all_manual.003" has been completed.
| Export multimanager...


| 0/9 - Exporting MultiManager content... - 00:00 < ?

| 0/3 - exporting quantity - 00:00 < ?

| Export multi.config.json for ac5ddb2b-7699-4134-be7e-b154bc0dfe8c


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.003/qurryinfo.json...
| Exporting qurrent.randomized_measure.003/qurryinfo.json done.


'ac5ddb2b-7699-4134-be7e-b154bc0dfe8c'

In [17]:
experiment_randomized.multimanagers[multi_exps1].quantity_container.keys()

dict_keys(['report.001', 'first_3.002', 'all_manual.003'])

In [18]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [ ]:
multi_exps1_reades = experiment_randomized.multiRead(
    save_location=".",
    summoner_name="qurrent.randomized_measure.001",
)

| Retrieve qurrent.randomized_measure.001...
| at: qurrent.randomized_measure.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

## Post-Process Availablities and Version Info

In [20]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  